In [ ]:
import rasterio
import geopandas as gpd
import pandas as pd
from rasterstats import point_query
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import xarray as xr
import rioxarray as rio
import rasterio
import pystac_client
import planetary_computer
from odc.stac import stac_load
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rioxarray as rio
from rasterio.features import geometry_mask
from shapely.geometry import Point
import matplotlib.pyplot as plt


In [ ]:
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)
bbox = [lower_left[1], lower_left[0], upper_right[1], upper_right[0]]  # [min lon, min lat, max lon, max lat]

# Define time range (June 1 – Sep 1, 2021)
time_window = "2012-06-01/2025-02-01"

In [ ]:
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

# Search for Sentinel-2 images with <30% cloud cover
search = stac.search(
    bbox=bbox,
    datetime=time_window,
    collections=["sentinel-2-l2a"],
    query={"eo:cloud_cover": {"lt": 10}},
)

# Get item list
items = list(search.get_items())
print(f"✅ Number of Sentinel-2 scenes found: {len(items)}")

# Sign URLs for access
signed_items = [planetary_computer.sign(item).to_dict() for item in items]

In [ ]:
# Define pixel resolution (10m per pixel)
resolution = 10  # meters
scale = resolution / 111320.0  # degrees per pixel for CRS=4326

# Load spectral bands from Sentinel-2
data = stac_load(
    items,
    bands=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12", "AOT", "WVP", "SCL","B09"],
    crs="EPSG:4326",  
    resolution=scale,  
    chunks={"df": 2048, "y": 2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bbox
)

print(data)

In [ ]:
median = data.median(dim="time").compute()

# Display an RGB Image (B04, B03, B02)
fig, ax = plt.subplots(figsize=(6, 6))
median[["B04", "B03", "B02"]].to_array().plot.imshow(robust=True, ax=ax, vmin=0, vmax=2500)
ax.set_title("RGB Median Composite (Cloud-Free)")
ax.axis('off')
plt.show()

In [ ]:
filename = "Sentinel2_NYC2.tif"

# Define raster transform
height, width = median.dims["latitude"], median.dims["longitude"]
transform = rasterio.transform.from_bounds(lower_left[1], lower_left[0], upper_right[1], upper_right[0], width, height)

# Write GeoTIFF
median.rio.write_crs("EPSG:4326", inplace=True)
median.rio.write_transform(transform, inplace=True)

with rasterio.open(filename, 'w', driver='GTiff', width=width, height=height,
                   crs="EPSG:4326", transform=transform,
                   count=len(median.data_vars), dtype="float32") as dst:
    for i, band in enumerate(median.data_vars.keys()):
        dst.write(median[band], i + 1)

print(f"✅ GeoTIFF saved: {filename}")